In [1]:
import pandas as pd
from stage import fullPrint
from datetime import date
import yahoo_fin.stock_info as yf
nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)

In [2]:
allList = []
nasdaqList = yf.tickers_nasdaq()
allList = (nasdaqList + nyseList)

In [3]:
SectorDict = {
    "Energy":"XLE",
    "Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Defensive":"XLP",
    "Healthcare":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrials":"XLI",
    "Utilities":"XLU",
    "Basic Materials":"XLB",
    "Financial Services":"XLF",
    "Real Estate":"XLRE"}
spList = yf.tickers_sp500()
spList = set(spList)
dictionary = {}
i = 0
allLen = str(len(nasdaqList))
for ticker in nasdaqList:
    try:
        df = yf.get_data(ticker, start_date="1995-01-02",end_date="2022-01-17", index_as_date = True, interval="1wk")
        df.to_pickle('stockData/nasdaq/'+ticker+'.pkl')
        industry = yf.get_company_info(ticker).at['sector','Value']
        dictionary[ticker] = SectorDict[industry]
    except:
        print(ticker+" not found")
    i = i+1
    print(str(i)+'/'+allLen)
import pickle
try:
    sectorDict = open('stockData/nasdaq.pkl', 'wb')
    pickle.dump(dictionary, sectorDict)
    sectorDict.close()
  
except:
    print("Something went wrong")

1/5490
2/5490
3/5490
AACIW not found
4/5490
5/5490
AADR not found
6/5490
7/5490
8/5490
9/5490
10/5490
11/5490
12/5490
13/5490
AAXJ not found
14/5490
15/5490
16/5490
17/5490
18/5490
19/5490
20/5490
21/5490
22/5490
23/5490
24/5490
25/5490
26/5490
27/5490
28/5490
29/5490
30/5490
31/5490
32/5490
ACAHW not found
33/5490
34/5490
35/5490
36/5490
ACBAW not found
37/5490
38/5490
39/5490
40/5490
41/5490
42/5490
43/5490
ACEVW not found
44/5490
45/5490
46/5490
47/5490
48/5490
49/5490
50/5490
51/5490
52/5490
ACKIT not found
53/5490
ACKIU not found
54/5490
ACKIW not found
55/5490
56/5490
57/5490
58/5490
59/5490
60/5490
61/5490
ACQRW not found
62/5490
63/5490
64/5490
65/5490
66/5490
67/5490
68/5490
ACTDW not found
69/5490
70/5490
71/5490
ACWI not found
72/5490
ACWX not found
73/5490
74/5490
75/5490
76/5490
77/5490
78/5490
79/5490
80/5490
81/5490
82/5490
ADERW not found
83/5490
84/5490
85/5490
86/5490
87/5490
ADILW not found
88/5490
89/5490
90/5490
91/5490
ADNWW not found
92/5490
93/5490
ADOCR not fou

In [3]:
import numpy as np
import yahoo_fin.stock_info as yf
weights = np.arange(1,31)
sumWeights = np.sum(weights)
spList = yf.tickers_sp500()
def calculateSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.iat[index,4]/df.iat[index-1,4]

def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iat[index,0]/df.iat[index-30,0]
def calculatePeak(df,index, diff, col):
    if index == 0:
        return df.iat[index,0]
    if index < diff:
        return max(df.iat[index-1,col],df.iat[index,0])


In [4]:
from stage import fullPrint
import swifter
import yahoo_fin.stock_info as yf
import pandas as pd
import os
spDf = yf.get_data('^GSPC', start_date="1995-01-02",end_date="2022-01-17", index_as_date = True, interval="1wk")
for ticker in allList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        if(stockDf['close'].isnull().values.any()):
            os.remove('stockData/nyseNasdaq/'+ticker+'.pkl')
            continue
        stockDf['fiveYearHigh'] = stockDf['close'].rolling(window=260).max().shift(1).fillna(0)  
        stockDf['WMA30'] = stockDf['close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
        stockDf['WMA30Slope'] = stockDf.swifter.apply(lambda x: calculateSlopeOfTicker(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['volumePerc'] = stockDf['volume'].pct_change()
        stockDf['percent'] = stockDf.swifter.apply(lambda x: product(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        spDf['percent'] = spDf.swifter.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
        stockDf['RS'] = stockDf['percent'] - spDf['percent']
        #stockDf = stockDf.dropna()
        stockDf['peak'] = 0
        stockDf['trough'] = 0
        stockDf['support'] = 0
        stockDf['initialSupport'] = 0
        for index, element in stockDf.iterrows():
            numIndex =stockDf.index.get_loc(index)
            if numIndex < 260:
                stockDf.iat[numIndex, 3] = calculatePeak(stockDf, numIndex, 260, 3)
        stockDf['Stage'] = ""
        stockDf['secondBuy'] = False
        print(ticker)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass

Pandas Apply: 100%|██████████| 1411/1411 [00:00<00:00, 24748.73it/s]


AACG


Pandas Apply: 100%|██████████| 1411/1411 [00:00<00:00, 24551.50it/s]


AACI


Pandas Apply: 100%|██████████| 1411/1411 [00:00<00:00, 23393.80it/s]


...

ZUO


Pandas Apply: 100%|██████████| 1411/1411 [00:00<00:00, 16749.15it/s]


ZWS


Pandas Apply: 100%|██████████| 1411/1411 [00:00<00:00, 21419.26it/s]


ZYME


In [ ]:
transactionTemplate = yf.get_data('AAPL', start_date="1995-01-02",end_date="2022-01-17", index_as_date = True, interval="1wk").drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
transactionTemplate.to_pickle("transactionTemplate.pkl")

In [10]:
pd.read_pickle('stockData/nyseNasdaq/AAPL.pkl').to_csv("AAPL.csv")

In [ ]:
for ticker in allList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        stockDf = stockDf.drop(['open','high','low','adjclose'],axis=1)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass